In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from pipelines.dig_a_plan_complete_master import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import change_schema_to_dig_a_plan_schema, duckdb_to_changes_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
change_schema: ChangesSchema = duckdb_to_changes_schema(file_path=".cache/input_data/boisy_grid.db")
# 2) build the *original* schema once

base_grid_data = change_schema_to_dig_a_plan_schema(change_schema = change_schema, s_base=1e6)

nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

max_n_trafo = 1
edge_id  = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)["edge_id"].to_list()
node_id = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)\
    .unpivot(on=["u_of_edge", "v_of_edge"])["value"].to_list()

base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(edge_id))
base_grid_data["node_data"] = base_grid_data["node_data"].filter(~c("node_id").is_in(node_id))
base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(
    pl.when(c("b_pu") < 1e-6).then(pl.lit(0)).otherwise(c("b_pu")).alias("b_pu"),
    pl.when(c("n_transfo") > max_n_trafo).then(pl.lit(max_n_trafo)).otherwise(c("n_transfo")).alias("n_transfo")
    )

base_grid_data["node_data"] = base_grid_data["node_data"].with_columns(
    (c("p_node_pu")*0.5*1e-3).alias("p_node_pu"),
    (c("p_node_pu")*0.5*1e-2).alias("q_node_pu"),
)

Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 214.52it/s]


In [3]:
base_grid_data["node_data"]["p_node_pu"].sum()*1e6

994.641609589041

In [4]:

dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e6, penalty_cost = 1e3, current_factor= 1e2, voltage_factor = 1e1, power_factor = 1e1,
    infeasibility_factor = 4
)

dig_a_plan.add_grid_data(**base_grid_data)


In [5]:
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: -2.4E+00, Slave obj: 2.5E-03 and Gap: 2.4E+00:   0%|          | 1/200 [00:21<1:10:57, 21.40s/it]


ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
dig_a_plan.slack_v_pos

N,slack_v_pos
i64,f64


In [ ]:
dig_a_plan.slack_v_neg

N,slack_v_neg
i64,f64
